Day 15: Beacon Exclusion Zone

In [34]:
import re
from itertools import combinations
from tqdm.notebook import tqdm

test_data = [line.strip() for line in open('Input/Day 15 Test.txt')]
real_data = [line.strip() for line in open('Input/Day 15.txt')]
data = real_data

# Part 1

In [35]:
# map out all beacons and the distance to their sensor

class Signal:
    def __init__(self, x, y, p, q):
        self.sensor_x = x
        self.sensor_y = y
        self.beacon_x = p
        self.beacon_y = q
        self.distance = abs(x - p) + abs(y - q)

    @property    
    def sensor(self):
        return (self.sensor_x, self.sensor_y)
    
    @property    
    def beacon(self):
        return (self.beacon, self.beacon_y)
    
        
    def __repr__(self):
        return f'Signal({(self.sensor_x, self.sensor_y)}, {(self.beacon_x, self.beacon_y)})'
    
    def exclusion(self, y):
        result = []
        x_min = self.sensor_x - (self.distance - abs(y - self.sensor_y))
        x_max = self.sensor_x + (self.distance - abs(y - self.sensor_y))
        return set(x for x in range(x_min, x_max + 1))
    
regex = re.compile(r'Sensor at x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)', re.I)
signals = []
for line in data:
    coords = [int(x) for x in regex.findall(line)[0]]
    signals.append(Signal(*coords))

scan_row = 2000000    
exclusion = set()
for signal in signals:
    exclusion |= (signal.exclusion(scan_row))
# remove beacon positions
for signal in signals:
    if signal.beacon_x in exclusion and signal.beacon_y == scan_row:
        exclusion.remove(signal.beacon_x)
    
print(len(exclusion))



4886370


signals

## Part 2
The idea is that the beacon is *just* outside the range of two sensors, and hence must be on the boundary of them

In [44]:
# start by determining which sensors have overlapping ranges
def distance(p1: (int, int), p2: (int, int)) -> int:
    return abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

def ring(center: (int, int), distance: int) -> set[int]:
    '''return a set of points at manhattan distance from center'''
    x, y = center
    result = { (x - distance, y), (x + distance, y) }
    for a in range(1 - distance, distance):
        b = abs(distance - a)
        result.add((x + a, y - b))
        result.add((x + a, y + b))
    return result

def crosspoints(s1, s2):
    CUTOFF = 4000000
    r1 = ring(s1.sensor, s1.distance + 1)
    r2 = ring(s2.sensor, s2.distance + 1)
    inter = r1 & r2
    return {p for p in inter if (p[0] >= 0) and (p[1] >= 0) and (p[0] <= CUTOFF) and (p[1] <= CUTOFF)}
 

In [39]:
investigate = set()
signal_combs = list(combinations(signals, 2))
for s1, s2 in tqdm(signal_combs):
    if distance(s1.sensor, s2.sensor) < (s1.distance + s2.distance):
        investigate.update(crosspoints(s1, s2))
print(len(investigate))

  0%|          | 0/703 [00:00<?, ?it/s]

2127776


Now, for each of these points investigate if their distance is outside *each* sensor range

In [42]:
for point in tqdm(investigate):
    for sensor, beacondist in [(s.sensor, s.distance) for s in signals]:
        if distance(sensor, point) <= beacondist:
            break
    else:
        break
print(point)

  0%|          | 0/2127776 [00:00<?, ?it/s]

(2843633, 2948438)


In [43]:
freq = point[0] * 4_000_000 + point[1]
print(freq)

11374534948438
